# `ospgrillage` Super-T Bridge Example

### Necessary imports

Working version for v0.1.0

In [11]:
import ospgrillage as og

### Visualizations
For the interaction 3D plots:
1. Install `node.js` from [here](https://nodejs.org/en/download/)
2. Install `ipympl`, e.g. `pip install ipympl`.
3. Install the jupyter-lab extensions:
```
jupyter labextension install @jupyter-widgets/jupyterlab-manager
jupyter labextension install jupyter-matplotlib
```
4. Decorate with the magic:

In [12]:
%matplotlib widget

ModuleNotFoundError: No module named 'ipympl'

### Units
Establish a consistent set of units - in this case the base units are N and m.

In [13]:
kilo = 1e3
milli = 1e-3
N = 1
m = 1
mm = milli * m
m2 = m ** 2
m3 = m ** 3
m4 = m ** 4
kN = kilo * N
MPa = N / ((mm) ** 2)
GPa = kilo * MPa

### Materials
Define materials

In [15]:
concrete = og.create_material(type="concrete", code="AS5100-2017", grade="65MPa")

### Cross Sections

In [16]:
edge_longitudinal_section = og.create_section(
    A=0.934 * m2,
    J=0.1857 * m3,
    Iz=0.3478 * m4,
    Iy=0.213602 * m4,
    Az=0.444795 * m2,
    Ay=0.258704 * m2,
)

longitudinal_section = og.create_section(
    A=1.025 * m2,
    J=0.1878 * m3,
    Iz=0.3694 * m4,
    Iy=0.113887e-3 * m4,
    Az=0.0371929 * m2,
    Ay=0.0371902 * m2,
)

transverse_section = og.create_section(
    A=0.504 * m2,
    J=5.22303e-3 * m3,
    Iy=0.32928 * m4,
    Iz=1.3608e-3 * m4,
    Ay=0.42 * m2,
    Az=0.42 * m2,
)

end_transverse_section = og.create_section(
    A=0.504 / 2 * m2,
    J=2.5012e-3 * m3,
    Iy=0.04116 * m4,
    Iz=0.6804e-3 * m4,
    Ay=0.21 * m2,
    Az=0.21 * m2,
)

### Grillage Members

In [17]:
longitudinal_beam = og.create_member(section=longitudinal_section, material=concrete)
edge_longitudinal_beam = og.create_member(
    section=edge_longitudinal_section, material=concrete
)
transverse_slab = og.create_member(section=transverse_section, material=concrete)
end_transverse_slab = og.create_member(
    section=end_transverse_section, material=concrete
)

### Bridge deck geometry

In [18]:
L = 33.5 * m  # span
w = 11.565 * m  # width
n_l = 7  # number of longitudinal members
n_t = 11  # number of transverse members
edge_dist = 1.05 * m  # distance between edge beam and first exterior beam
angle = 0  # skew angle

## Create the Grillage Model

In [19]:
model = og.create_grillage(
    bridge_name="Super-T 33_5m",
    long_dim=L,
    width=w,
    skew=angle,
    num_long_grid=n_l,
    num_trans_grid=n_t,
    edge_beam_dist=edge_dist,
)

# assign grillage member to element groups of grillage model
model.set_member(longitudinal_beam, member="interior_main_beam")
model.set_member(longitudinal_beam, member="exterior_main_beam_1")
model.set_member(longitudinal_beam, member="exterior_main_beam_2")
model.set_member(edge_longitudinal_beam, member="edge_beam")
model.set_member(transverse_slab, member="transverse_slab")
model.set_member(end_transverse_slab, member="start_edge")
model.set_member(end_transverse_slab, member="end_edge")

Edge mesh @ start span completed
Orthogonal meshing complete
Setting member: interior_main_beam of model
Setting member: exterior_main_beam_1 of model
Setting member: exterior_main_beam_2 of model
Setting member: edge_beam of model
Setting member: transverse_slab of model
Setting member: start_edge of model
Setting member: end_edge of model


### Plot the model
To do  this, we pass the model into OpenSeesPy to use its visualization functions

Note: we may need to add some depdencies to view these...https://code-examples.net/en/q/2fd38ba

In [20]:
model.create_osp_model(pyfile=False)

In [21]:
# plotting using the opsplt library
og.opsplt.plot_model("nodes");

No Model_ODB specified, trying to get data from the active model.
3D model


<IPython.core.display.Javascript object>

In [22]:
# Using the opsvis library
og.opsv.plot_model(az_el=(-90, 0));

<IPython.core.display.Javascript object>

## Load cases
Here, we'll just set some names in a list that we'll refer to later, and also define a reference point load:

In [23]:
# reference unit load for various load types
P = 1 * kN

# name strings of load cases to be created
static_loadcase_names = [
    "Line Test Case",
    "Points Test Case (Global)",
    "Points Test Case (Local in Point)",
    "Patch Test Case",
]

### Load case 1
Line load running along midspan width (P is kN/m)

In [25]:
# Create load vertices in global coordinate system
line_vertex_1 = og.create_load_vertex(x=L / 2, z=0, p=P)
line_vertex_2 = og.create_load_vertex(x=L / 2, z=w, p=P)
test_line_load = og.create_load(
    type="line", name="Test Load", point1=line_vertex_1, point2=line_vertex_2
)

# Create load case, add loads, and assign to model
line_case = og.create_load_case(name=static_loadcase_names[0])
line_case.add_load(test_line_load)
model.add_load_case(line_case)

Load Case: Line Test Case added


### Load case 2
Compound point loads along midspan width (P is kN). 

Note: The compound load in this load case is directly assigned wrt the global grillage coordinate

In [26]:
# working in global coordinate system
load_positions = [
    0,
    edge_dist,
    edge_dist + 2 * m,
    edge_dist + 4 * m,
    edge_dist + 6 * m,
    w - edge_dist,
    w,
]  # creating list of load position

test_points_load = og.create_compound_load(name="Points Test Case (Global)")

# create point load in global coordinate
for pos in load_positions:
    point = og.create_load(
        type="point", name="Point", point1=og.create_load_vertex(x=L / 2, z=pos, p=P)
    )
    # add to compound load
    test_points_load.add_load(load_obj=point)

# Create load case, add loads, and assign
points_case = og.create_load_case(name=static_loadcase_names[1])
points_case.add_load(test_points_load)
model.add_load_case(points_case)

Load Case: Points Test Case (Global) added


### Load case 3
Compound point loads along midspan width

Note: This is a version of Load case 2 but demonstrating the steps of setting up a compound load in a local coordinate (say for a moving truck), and then assigning it to the global grillage coordinate system.

**CC** - how is the global when the z is still from the list above?

**JN** - In Load case 3, the x of the compound load (local) is 0, then `set_global_coord()` shift the origin by (L/2,0,0) , with z being the same as that of the global coordinate. 

In [27]:
# working in user-defined local coordinate (in point load)
test_points_load = og.create_compound_load(name="Points Test Case (Local in Point)")

# create point load in local coordinate space
for pos in load_positions:
    point = og.create_load(
        type="point", name="Point", point1=og.create_load_vertex(x=0, z=pos, p=P)
    )
    # add to compound load
    test_points_load.add_load(load_obj=point)

# shift from local to global
test_points_load.set_global_coord(og.Point(L / 2, 0, 0))

# Create load case, add loads, and assign
points_case = og.create_load_case(name=static_loadcase_names[2])
points_case.add_load(test_points_load)
model.add_load_case(points_case)

Load Case: Points Test Case (Local in Point) added


### Load case 4
Patch load over entire bridge deck (P is kN/m2)

In [28]:
patch_vertex_1 = og.create_load_vertex(x=0, z=0, p=P)
patch_vertex_2 = og.create_load_vertex(x=L, z=0, p=P)
patch_vertex_3 = og.create_load_vertex(x=L, z=w, p=P)
patch_vertex_4 = og.create_load_vertex(x=0, z=w, p=P)
test_patch_load = og.create_load(
    type="patch",
    name="Test Load",
    point1=patch_vertex_1,
    point2=patch_vertex_2,
    point3=patch_vertex_3,
    point4=patch_vertex_4,
)

# Create load case, add loads, and assign
patch_case = og.create_load_case(name=static_loadcase_names[3])
patch_case.add_load(test_patch_load)
model.add_load_case(patch_case)

Load Case: Patch Test Case added


c:\users\nicni\pycharmprojects\ospg\lib\site-packages\scipy\optimize\minpack.py:175: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  warnings.warn(msg, RuntimeWarning)


### Moving load case

#### Truck
2 axle truck (equal loads, 2x2 spacing centre line running)

In [29]:
# create truck as a compound load in a local coordinate system
two_axle_truck = og.create_compound_load(name="Two Axle Truck")
axl_w = 2 * m  # axle width
axl_s = 2 * m  # axle spacing
veh_l = axl_s  # vehicle length

# note here we show that we can directly interact and create load vertex 
# using LoadPoint namedtuple instead of create_load_vertex()
point1 = og.create_load(
    type="point", name="Point", point1=og.LoadPoint(x=0, y=0, z=0, p=P)
)
point2 = og.create_load(
    type="point", name="Point", point1=og.LoadPoint(x=0, y=0, z=axl_w, p=P)
)
point3 = og.create_load(
    type="point", name="Point", point1=og.LoadPoint(x=axl_s, y=0, z=axl_w, p=P)
)
point4 = og.create_load(
    type="point", name="Point", point1=og.LoadPoint(x=axl_s, y=0, z=0, p=P)
)

two_axle_truck.add_load(load_obj=point1)
two_axle_truck.add_load(load_obj=point2)
two_axle_truck.add_load(load_obj=point3)

two_axle_truck.add_load(load_obj=point4)

#### Moving load path
Create the path object in global coordinate system - centre line running of entire span.
When local coord: the path describes where the moving load *origin* is to start and end.

The path outlines the position of the origin of compound load (local) - it shall move from `start_point` to `end_point` within some
specified `increments`. 

In this example, we move the truck as shown in Figure #

[FIGURE HERE]

CC - what does this last sentence mean?

JN - it is saying, the origin of the moving load (or moving compound load) moves from `start_point` to `end_point` with `increments`. 

In [30]:
single_path = og.create_moving_path(
    start_point=og.Point(0 - axl_w, 0, w / 2 - axl_w / 2),
    end_point=og.Point(L, 0, w / 2 - axl_w / 2),
    increments=int(L + veh_l + 1),
)

#### Moving load case

In [31]:
# create moving load (and case)
moving_truck = og.create_moving_load(name="Moving Two Axle Truck")

# Set path to all loads defined within moving_truck
moving_truck.set_path(single_path)

# note: it is possible to set different paths for different compound loads in one moving load object
moving_truck.add_load(two_axle_truck)
model.add_load_case(moving_truck)

Moving load case: Moving Two Axle Truck created


## Analysis

In [32]:
model.analyze();

Analysis: Line Test Case completed
Extraction of results completed for Analysis:Line Test Case 
Analysis: Line Test Case completed
Extraction of results completed for Analysis:Line Test Case 
Analysis: Points Test Case (Global) completed
Extraction of results completed for Analysis:Points Test Case (Global) 
Analysis: Points Test Case (Local in Point) completed
Extraction of results completed for Analysis:Points Test Case (Local in Point) 
Analysis: Patch Test Case completed
Extraction of results completed for Analysis:Patch Test Case 
Analysis: Moving Two Axle Truck at global position [-2.00,0.00,4.78] completed
Extraction of results completed for Analysis:Moving Two Axle Truck at global position [-2.00,0.00,4.78] 
Analysis: Moving Two Axle Truck at global position [-0.99,0.00,4.78] completed
Extraction of results completed for Analysis:Moving Two Axle Truck at global position [-0.99,0.00,4.78] 
Analysis: Moving Two Axle Truck at global position [0.03,0.00,4.78] completed
Extraction o

## Results

Extract the results - `xarray` format

In [33]:
results = model.get_results()

## Post-Processing

### Load combination
Create a dictionary with the loadcase names and their respective load factors to get the load combination results:

In [34]:
l_factor = 2.3
p_factor = 0.5
load_combinations = {static_loadcase_names[0]:l_factor,
                     static_loadcase_names[-1]:p_factor}

combination_results = model.get_results(combinations=load_combinations)

Obtaining load combinations ....


In [35]:
combination_results.forces.sel(Component="Mz_i",Element=list(range(84, 90 + 1))).sum()

<xarray.DataArray 'forces' ()>
array(1039838.95312471)
Coordinates:
    Component  <U7 'Mz_i'

### Results extraction

In [36]:
results

<xarray.Dataset>
Dimensions:        (Component: 18, Loadcase: 40, Node: 77, Element: 136, Nodes: 2)
Coordinates:
  * Component      (Component) <U7 'Mx_i' 'Mx_j' 'My_i' ... 'theta_y' 'theta_z'
  * Loadcase       (Loadcase) <U58 'Line Test Case' ... 'Moving Two Axle Truc...
  * Node           (Node) int32 1 2 3 4 5 6 7 8 9 ... 69 70 71 72 73 74 75 76 77
  * Element        (Element) int32 1 2 3 4 5 6 7 ... 130 131 132 133 134 135 136
  * Nodes          (Nodes) <U1 'i' 'j'
Data variables:
    displacements  (Loadcase, Node, Component) float64 nan nan ... 0.0 -4.14e-20
    forces         (Loadcase, Element, Component) float64 -284.3 -286.8 ... nan
    ele_nodes      (Element, Nodes) int32 1 2 2 3 3 4 4 ... 11 75 12 76 13 77 14

In [43]:
# get list of longitudinal element tags along/near mid_span i.e. 84 to 90 in Figure 1
ele_set = list(range(84, 90 + 1))
print(ele_set)
# query
extracted_bending = results.forces.sel(Loadcase=static_loadcase_names, Element=ele_set, Component="Mz_i")


[84, 85, 86, 87, 88, 89, 90]


In [44]:
import numpy as np
np.sum(
np.array(
    results.forces.sel(
        Loadcase=static_loadcase_names, Element=ele_set, Component="Mz_i"
    )
),
axis=1,
)

array([  96856.87499998,   58625.00000001,   58625.        ,
       1634136.28124952])

### Moving load results

In [45]:
# call the results and
move_results = model.get_results(load_case="Moving Two Axle Truck")

Extracted moving load case results : Moving Two Axle Truck


In [46]:
# selecting load case of specific load position
integer = int(
    L / 2 - 1 + 2
)  # here we choose when the load groups are at/near mid span L = 14m i.e. 17

# query
mid_span_bending = move_results.forces.isel(Loadcase=integer).sel(
    Element=ele_set, Component="Mz_i"
)

In [47]:
bending_z = np.sum(np.array(mid_span_bending))

# Hand calc:
bending_z_theoretical = 2*P*(L/2-axl_s/2) # 31500

print("bending_z ={}".format(bending_z))
print("bending_z_theoretical ={}".format(bending_z_theoretical))

bending_z =31499.999999999927
bending_z_theoretical =31500.0


In [48]:
og.plot_force(model, results, member="exterior_main_beam_2", component="Mz")

<IPython.core.display.Javascript object>